In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class JoukowskiTranformation:

    def __init__(self, center, c_constant = 1.0) -> None:
        self.center = center
        self.c_constant = c_constant
        self.radius = np.absolute( 1.0 - self.center )

    def transform(self, z_prime):
        z = z_prime
        z = z + self.center
        return z + self.c_constant**2 / z
    
    def inverse_transform(self, z):
        r1 = 0.5 * ( z + np.sqrt(z**2 - 4 * self.c_constant**2) )
        r2 = 0.5 * ( z - np.sqrt(z**2 - 4 * self.c_constant**2) )

        r1 = r1 - self.center
        r2 = r2 - self.center

        z_prime = np.where(np.absolute(r1) > np.absolute(r2), r1, r2)

        return z_prime

    def derivative(self, z_prime):
        z = z_prime
        z = z + self.center
        derivative = 1 - self.c_constant**2 / z**2
        return derivative


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class ParallelFlow:

    def __init__(self, velocity_x: float, velocity_y: float) -> None:
        """Initializer for the parallel flow class. Stores relevant quantities as class members.

        Args:
            velocity_x (float): Cartesion component of the velocity pointing in x-direction.
            velocity_y (float): Cartesion component of the velocity pointing in y-direction.
        """
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Return the complex velocity of the parallel flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        return self.velocity_x - 1.0j * self.velocity_y

class DipolFlow:

    def __init__(self, dipol_strength, translation_x = 0.0, translation_y = 0.0) -> None:
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.dipol_strength = dipol_strength
        self.factor = self.dipol_strength / np.pi

    def get_complex_velocity(self, z):
        z = z - self.translation_x - 1.0j * self.translation_y
        return - self.factor / (z*z)

class PotentialVortexFlow:

    def __init__(self, gamma: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the vortex flow class. Stores relevant quantities as class members.

        Args:
            gamma (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.gamma = gamma
        self.factor = self.gamma / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the vortex flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z = z - self.translation_x - 1.0j * self.translation_y
        return - 1.0j * self.factor / z

class SourceFlow:

    def __init__(self, source_strength: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the source- and sink-flow class. Stores relevant quantities as class members.

        Args:
            source_strength (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.source_strength = source_strength
        self.factor = self.source_strength / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the source-/sink-flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z = z - self.translation_x - 1.0j * self.translation_y
        return self.factor / z
    
def get_grid_quantities(x_min: float, x_max: float, y_min: float, y_max: float, resolution_x: int, resolution_y: int):
    """Returns the meshgrid used for the calculations.

    Args:
        x_min (float): The minimum x value.
        x_max (float): The maximum x value.
        y_min (float): The minimum y value.
        y_max (float): The maximum y value.
        resolution_x (int): The number of points to discretize in x direction.
        resolution_y (int): The number of points to discretize in y direction.

    Returns:
        List[np.array]: The numpy array containing the meshgrid.
    """
    x_linspace = np.linspace(x_min, x_max, resolution_x)
    y_linspace = np.linspace(y_min, y_max, resolution_y)
    X,Y = np.meshgrid(x_linspace, y_linspace)
    Z = X + 1.0j * Y
    return X, Y, Z

In [ ]:
x_min = -3.0
x_max = 3.0
y_min = -3.0
y_max = 3.0
resolution_per_unit_length = 20
resolution_x = ( x_max - x_min ) * resolution_per_unit_length
resolution_y = ( y_max - y_min ) * resolution_per_unit_length

X, Y, Z = get_grid_quantities(x_min, x_max, y_min, y_max, int(resolution_x), int(resolution_y))

In [ ]:


center = -0.0 + 0.0j
joukowski_transformation = JoukowskiTranformation(center)

phi_linspace = np.linspace(0.0, 2.0*np.pi, 100)
radius = joukowski_transformation.radius
print(f"Radius: {radius}")

x_circle = radius * np.cos(phi_linspace)
y_circle = radius * np.sin(phi_linspace)
z_circle = x_circle + 1.0j * y_circle

z_prime = z_circle

z_foil = joukowski_transformation.transform(z_prime)
z_retransformed = joukowski_transformation.inverse_transform(z_foil)

plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

plt.scatter(np.real(z_prime), np.imag(z_prime), c="red")
plt.scatter(np.real(z_foil), np.imag(z_foil), c="green")
plt.scatter(np.real(z_retransformed), np.imag(z_retransformed), c="blue", marker="x")

plt.show()



In [ ]:
z_meshgrid = np.where(np.absolute(Z) <= joukowski_transformation.radius, np.nan, Z)
z_prime_meshgrid = z_meshgrid
z_foil_meshgrid = joukowski_transformation.transform(z_prime_meshgrid)
inverse_meshgrid = joukowski_transformation.inverse_transform(z_foil_meshgrid)

plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

plt.scatter(np.real(z_prime_meshgrid), np.imag(z_prime_meshgrid), c="red", s=0.1)
plt.scatter(np.real(z_foil_meshgrid), np.imag(z_foil_meshgrid), c="green", s=0.1)
# plt.scatter(np.real(z_retransformed), np.imag(z_retransformed), c="blue", marker="x")

plt.show()

In [ ]:
U_infty = 1.0
V_infty = 0.0
total_inflow_velocity = np.sqrt(U_infty**2 + V_infty**2)
radius = joukowski_transformation.radius
dipole_strength = np.pi * radius**2 * total_inflow_velocity

small_gamma = - np.imag(joukowski_transformation.center) / radius - (V_infty / total_inflow_velocity)
gamma = 4.0 * np.pi * total_inflow_velocity * radius * small_gamma

parallel_flow = ParallelFlow(U_infty, V_infty)
dipole_flow = DipolFlow(dipole_strength, 0.0, 0.0)
vortex_flow = PotentialVortexFlow(gamma, 0.0, 0.0)


Z_inverse = joukowski_transformation.inverse_transform(Z)
Z_inverse = np.where(np.absolute(Z_inverse) <= joukowski_transformation.radius, np.nan, Z_inverse)

complex_velocity = parallel_flow.get_complex_velocity(Z_inverse) + dipole_flow.get_complex_velocity(Z_inverse) + vortex_flow.get_complex_velocity(Z_inverse)
complex_velocity = complex_velocity / joukowski_transformation.derivative(Z_inverse)

u = np.real(complex_velocity)
v = - np.imag(complex_velocity)

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')

x_linspace = np.linspace(x_min, x_max, 21)
y_linspace = np.linspace(x_min, x_max, 50)

# Plot the streamlines.

# The streamline seeds describe all the points where streamlines start. Thus, only streamline the go through these points are drawn.
# This may heavily influence the look of the plot!
# In this case only points to the left (*xmin refers to the left, replace with e.g. 0.0 to also obtain streamlines inside the oval)
# of the oval are choosen. Thus, no streamlines inside the oval show.
# In this case, they are equally distributed along the y direction. To also indicate the stagnation streamlines, values slightly above
# and below (+/- 1.e-5) the x axis are added.
streamline_seeds = np.stack([np.ones_like(y_linspace)*x_min, y_linspace], axis=-1)

ax.streamplot(
    X, Y, u, v,
    density=[1.0, 1.0],
    broken_streamlines=False,
    linewidth=0.6,
    # start_points=streamline_seeds,
    color="#3070b3", # This beautiful color is TUMBlue
)

ax.streamplot(
    X, Y, -v, u,
    density=[0.2, 0.2],
    broken_streamlines=False,
    arrowstyle = "-",
    linewidth=0.6,
    color="#E37222", # This beautiful color is TUMOrange
)

# plt.quiver(X, Y, u, v)

plt.plot(np.real(z_foil), np.imag(z_foil), linewidth=1.6, color="#A2AD00")

# plt.scatter(np.real(z_foil), np.imag(z_foil), c=np.absolute(complex_velocity), s=0.1)
# plt.quiver(np.real(z_foil), np.imag(z_foil), u, v)

plt.savefig("foil.pdf")
plt.show()